In [7]:
#figure out what files were skipped for/are missing from results

import os, shutil
import time
from os import listdir
from os.path import isfile, join


direct2= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human/splits/'
new_splits= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human/splits2/'
direct3= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_results/'
files = [f for f in listdir(direct2) if isfile(join(direct2,f)) ]
results =  [f for f in listdir(direct3) if isfile(join(direct3,f)) ]
files.sort()
pot_res=[]
for i in range(len(files)):
    temp='results'+str(i)+'.csv'
    pot_res.append(temp)
missing = [name for name in pot_res if name not in results]
# print(missing)
filestorun=[]
for i in missing:
    temp= i[:-4]
    temp=int(temp[7:])
    filestorun.append(files[temp])

for i in filestorun:
    # print(i)
    full_file_name = os.path.join(direct2,i)
    # print(full_file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name,new_splits)

# print(files)
# print(results)

In [5]:

import os, shutil
import time
from os import listdir
from os.path import isfile, join
new_splits= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human/splits2/'
filestorun = [f for f in listdir(new_splits) if isfile(join(new_splits,f)) ]
print(filestorun)

['human01.fasta']


In [14]:
#60 sequences per line
# max of 1000 sequences per entry
# max of 16 lines
#take each file, split proteins down to sequences of less than 1000 and make each of them a new file
new_splits= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human/splits4/'
split_splits='C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human/splits5/'
filestorun = [f for f in listdir(new_splits) if isfile(join(new_splits,f)) ]
print(filestorun)
split_lines=14
linemax=250
string1 = '>'
i=1000
#step through current splits of the original fasta file and separate them into smaller splits if need be
for file in filestorun:
    lineindex = 0
    protind=0


    tempfile = open(new_splits+file, 'r')
    tempnew = open(split_splits+'human'+str(i)+'.fasta', 'w')


    for line in tempfile:
        lineindex +=1
        protind +=1
        if string1 in line:
            protind=0
            header=line
        if protind>split_lines:
            i += 1
            lineindex = 0
            tempnew.close()
            tempnew = open(split_splits+'human'+str(i)+'.fasta', 'w')
            tempnew.write(header)
        if lineindex == linemax:
            i += 1
            lineindex = 0
            tempnew.close()
            tempnew = open(split_splits+'human'+str(i)+'.fasta', 'w')
            tempnew.write(header)
        tempnew.write(line)
    tempfile.close()
    tempnew.close()
    i+=1
    print(i)

['human01.fasta', 'human1039.fasta', 'human1040.fasta', 'human1041.fasta', 'human1047.fasta', 'human1155.fasta', 'human1163.fasta', 'human1169.fasta', 'human1277.fasta', 'human129.fasta', 'human1298.fasta', 'human1303.fasta', 'human1341.fasta', 'human1343.fasta', 'human137.fasta', 'human1382.fasta', 'human1386.fasta', 'human1434.fasta', 'human1449.fasta', 'human1462.fasta', 'human1465.fasta', 'human1486.fasta', 'human1529.fasta', 'human1530.fasta', 'human1533.fasta', 'human1534.fasta', 'human1571.fasta', 'human1635.fasta', 'human1656.fasta', 'human1692.fasta', 'human1718.fasta', 'human1824.fasta', 'human1827.fasta', 'human1828.fasta', 'human1836.fasta', 'human1855.fasta', 'human189.fasta', 'human1924.fasta', 'human197.fasta', 'human1970.fasta', 'human1981.fasta', 'human1987.fasta', 'human200.fasta', 'human2000.fasta', 'human2001.fasta', 'human2002.fasta', 'human201.fasta', 'human202.fasta', 'human203.fasta', 'human205.fasta', 'human206.fasta', 'human207.fasta', 'human208.fasta', 'human

In [15]:
#cleanup output csvs
#any protein with a comma in its name is messed up
#pattern if the line does not end with a comma then remove the first comma after opening as a txt
import os
import time
from os import listdir
from os.path import isfile, join
# file1=open('results0_test.csv','r')
# file2=open('results0_clean.csv', 'w')
direct1= 'Human_clean_results/'
direct2='Human_clean_results/Cleaner_results/'
files = [f for f in listdir(direct1) if isfile(join(direct1,f)) ]
for i in range(len(files)):
    file1=open(direct1+files[i],'r')
    file2=open(direct2+'results'+str(i)+'.csv.','w')
    for line in file1:
#     print(line.count(','))
        while line.count(',')>20:
            line=line.replace(',','',1)
            print(line.count(','))
        file2.write(line)
    file1.close()
    file2.close()

['human1039.fasta',
 'human1040.fasta',
 'human1041.fasta',
 'human1047.fasta',
 'human1155.fasta',
 'human1163.fasta',
 'human1169.fasta']